In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/LTI_Pole_Estimation.git -b casestudies

# # To update the work environment
# %cd /content/drive/My Drive/LTI_Pole_Estimation
# !git pull

%cd /content/drive/My Drive/LTI_Pole_Estimation/casestudies

In [ ]:
import os
import sys
sys.path.append("../framework/")
sys.path.append("../sl/")
sys.path.append("../pole/")

import numpy as np
import matplotlib.pylab as plt
import pandas as pd

from pole_environment import PoleEnvironment
from pole_environment_factory import PoleEnvironmentFactory
from pole_build_parameter import PoleBuildParameter
from pole_build_parameter_factory import PoleBuildParameterFactory
from builder import Builder
from store import Store
from pole_agent_factory import PoleAgentFactory
from pole_trainer_factory import PoleTrainerFactory
from mylogger import MyLogger

from loader import Loader

In [ ]:
case_study_name = "case_study_001a"

In [ ]:
trainingHistoryDbPath = "%s_training_history.sqlite" % case_study_name

## S400: Load trained agents to analyze them

### SS410: choose a trained agent

In [ ]:
agentFactory = PoleAgentFactory()
environmentFactory = PoleEnvironmentFactory()

store = Store(trainingHistoryDbPath)

buildParameterFactory = PoleBuildParameterFactory()
loader = Loader(agentFactory=agentFactory
                , environmentFactory=environmentFactory
                , buildParameterFactory=buildParameterFactory
                , store = store)

In [ ]:
def genTrainedAgents():
    for agent, buildParameter, epoch in loader.load("{0}%".format(case_study_name)):
        yield agent, buildParameter, epoch

### SS420: check the distribution of poles

In [ ]:
def show_poles(agent, environment):

    eig_true = environment.get_eig() # (nHiddeEnvironment,)
    eig_estimated = agent.get_eig() # (nHiddenAgent,)

    fig = plt.gcf()

    th = np.linspace(0,1,2**6)*2*np.pi
    ax = fig.add_subplot()
    ax.plot(np.cos(th), np.sin(th), 'k--')
    ax.axhline(0, color="k", linestyle='--')
    ax.axvline(0, color="k", linestyle='--')
    ax.plot(eig_true.real, eig_true.imag, 'x', label = "true")
    ax.plot(eig_estimated.real, eig_estimated.imag, 'x', label = "estimated")
    ax.legend()
    plt.tight_layout()

In [ ]:
triplets = [*genTrainedAgents()]

for k1 in np.random.choice(len(triplets), size=(3,)):
    agent, buildParameter, epoch = triplets[k1]
    environment = environmentFactory.create(buildParameter)
    
    fig = plt.figure(figsize=[10/2.57, 10/2.57])
    show_poles(agent, environment)

### SS430: check the discrepancy of estimated poles from true poles

In [ ]:
def evaluatePolesDiscrepancy(agent, environment):
    eig_true = environment.get_eig() # (nHiddeEnvironment,)
    eig_estimated = agent.get_eig() # (nHiddenAgent,)
    gap = eig_true.reshape(-1,1) - eig_estimated # (nHiddenEnvironment, nHiddenAgent)
    gap_total = np.mean(np.min(np.abs(gap), axis=1)) # (,)
    return gap_total # (,)

In [ ]:
tbl = {
    "epoch": []
    , "gap_total": []    
    , **{key: [] for key in buildParameter.__dict__}
    }

for agent, buildParameter, epoch in genTrainedAgents():
    environment = environmentFactory.create(buildParameter)
    gap_total = evaluatePolesDiscrepancy(agent, environment)
    
    tbl["epoch"].append(epoch)
    tbl["gap_total"].append(gap_total)
    for key in buildParameter.__dict__:
        tbl[key].append(buildParameter.__dict__[key])
tbl = pd.DataFrame(tbl)

In [ ]:
t = []
x = []
for epoch in np.unique(tbl.epoch):
    idx = tbl.epoch == epoch
    x.append(tbl.gap_total[idx])
    t.append(epoch)
plt.boxplot(x, labels=t)
plt.xlabel('epoch')
plt.title('discrepancy')
pass

export the table to csv file:

In [ ]:
tbl.to_csv("%s_discrepancy.csv" % case_study_name)

### SS440: check the distribution of poles

In [ ]:
def export_pole_distribution(itr):

    for k1, (agent, buildParameter, epoch) in enumerate(itr):
        if k1 == 0:
            tbl = {
                "epoch": []
                , "pole.type": []
                , "real": []
                , "imag": []
                , **{key: [] for key in buildParameter.__dict__}
                }        
        
        environment = environmentFactory.create(buildParameter)

        eig_true = environment.get_eig()
        eig_true = np.stack((eig_true.real
            , eig_true.imag), axis=-1)

        eig_estimated = agent.get_eig()
        eig_estimated = np.stack((eig_estimated.real
            , eig_estimated.imag), axis=-1)

        for vec in eig_true:
            tbl["pole.type"].append("true")
            tbl["real"].append(vec[0])
            tbl["imag"].append(vec[1])
            tbl["epoch"].append(epoch)
            for key in buildParameter.__dict__:
                tbl[key].append(buildParameter.__dict__[key])

        for vec in eig_estimated:
            tbl["pole.type"].append("estimated")
            tbl["real"].append(vec[0])
            tbl["imag"].append(vec[1])
            tbl["epoch"].append(epoch)        
            for key in buildParameter.__dict__:
                tbl[key].append(buildParameter.__dict__[key])

    tbl = pd.DataFrame(tbl)

    #export the table to csv file:
    tbl.to_csv("%s_pole_distribution.csv" % case_study_name)

In [ ]:
export_pole_distribution(genTrainedAgents())